# Tutorial: Describe Neuroscience Dataset using MINDS

## Initialize and configure

In [ ]:
!pip install nexusforge==0.6.2

In [ ]:
!pip install allensdk

In [ ]:
!pip install neurom[plotly]

In [ ]:
!pip install --upgrade nest-asyncio==1.5.1

### Get an authentication token

The [Nexus sandbox application](https://sandbox.bluebrainnexus.io/web) can be used to get a token:

- Step 1: From the [web page](https://sandbox.bluebrainnexus.io/web), click on the login button in the top right corner and follow the instructions on screen.

- Step 2: You will then see a `Copy token` button in the top right corner. Click on it to copy the token to the clipboard.


Once a token is obtained, proceed to paste it as the value of the `TOKEN` variable below.

__Important__: A Nexus token is valid for 8 hours, if your working session is open for more than 8 hours, you may need to refresh the value of the token and reintialize the forge client in the _'Configure a forge client to store, manage and access datasets'_ section below. 

In [ ]:
import getpass

In [ ]:
TOKEN = getpass.getpass()

### Configure a forge client to store, manage and access datasets

In [ ]:
import uuid
import base64
import requests
import json
from pathlib import Path

from kgforge.core import KnowledgeGraphForge
from kgforge.specializations.mappings import DictionaryMapping

from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.core.cell_types_cache import CellTypesCache

In [ ]:
r = requests.get('https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/rdfmodel/jsonldcontext.json')
dirpath = './rdfmodel'
Path(dirpath).mkdir(parents=True, exist_ok=True)
with open(f'{dirpath}/jsonldcontext.json', 'w') as outfile:
    json.dump(r.json(), outfile)

In [ ]:
ORG = "github-users"
PROJECT = ""  # Provide here the automatically created project name created when you logged into the Nexus sandbox instance.

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/forge.yml",
                            bucket=f"{ORG}/{PROJECT}",
                            endpoint="https://sandbox.bluebrainnexus.io/v1",
                            token=TOKEN)

## Download datasets from Allen Cell Types Database and from MouseLight

### Download mouse neuron morphology from the Allen Cell Types Database

We will be downloading mouse neuron morphology data from the [Allen Cell Types Database](https://celltypes.brain-map.org/). The [AllenSDK](https://allensdk.readthedocs.io/en/latest/) can be used for data download.

In [ ]:
ALLEN_DIR = "allen_cell_types_database"

In [ ]:
ctc = CellTypesCache(manifest_file=f"{ALLEN_DIR}/manifest.json")

In [ ]:
MAX_CELLS = 1
SPECIES = CellTypesApi.MOUSE

In [ ]:
nm_allen_identifiers = [cell["id"] for cell in ctc.get_cells(species=[SPECIES], require_reconstruction = True)][:MAX_CELLS]
print(f"Selected a mouse neuron with identifier: {nm_allen_identifiers}")

In [ ]:
with open(f"{ALLEN_DIR}/cells.json") as f:
    allen_cell_types_metadata = json.load(f)

In [ ]:
nm_allen_metadata = [neuron for neuron in allen_cell_types_metadata if neuron["specimen__id"] in nm_allen_identifiers]

In [ ]:
print(f"Metadata of the neuron {nm_allen_identifiers}:")
nm_allen_metadata

### Download one mouse neuron morphology reconstructed from the selected neuron

We will be downloading one mouse neuron morphology from the [Allen Cell Types Database](https://celltypes.brain-map.org/) using the [AllenSDK](https://allensdk.readthedocs.io/en/latest/).

In [ ]:
for identifier in nm_allen_identifiers:
    ctc.get_reconstruction(identifier)

### Download one mouse neuron electrophysiology recording from the selected neuron

We will be downloading one mouse neuron electrophysiology from the [Allen Cell Types Database](https://celltypes.brain-map.org/) using the [AllenSDK](https://allensdk.readthedocs.io/en/latest/).

In [ ]:
for identifier in nm_allen_identifiers:
    ctc.get_ephys_data(identifier)

## Transform Allen Cell Types Database Metadata to [Neuroshapes' MINDS](https://bbp-nexus.epfl.ch/datamodels/class-schemadataset.html) metadata

### Map the Allen Cell Types Database neuron morphologies metadata to Neuroshapes

In [ ]:
allen_nm_mapping = DictionaryMapping.load("https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/mappings/allen_morphology_dataset.hjson")
nm_allen_resources = forge.map(nm_allen_metadata, allen_nm_mapping, na='')

### Map the Allen Cell Types Database neuron electrophysiology recording to Neuroshapes

In [ ]:
allen_ephys_mapping = DictionaryMapping.load("https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/mappings/allen_ephys_dataset.hjson")
nephys_allen_resources = forge.map(nm_allen_metadata, allen_ephys_mapping, na='')

## Register

If the registration fails, try refreshing the access token and reinitializing the forge client in the _'Configure a forge client to store, manage and access datasets'_ section.

### Register the Allen Cell Types Database neuron morphology

In [ ]:
nm_allen_resources.id = forge.format("identifier", "neuronmorphologies", str(uuid.uuid4()))

In [ ]:
forge.register(nm_allen_resources)

### Register the Allen Cell Types Database neuron electrophysiology recording

In [ ]:
nephys_allen_resources.id = forge.format("identifier", "traces", str(uuid.uuid4()))

In [ ]:
forge.register(nephys_allen_resources)

## Access

### Set filters

In [ ]:
_type = "NeuronMorphology"

filters = {"type": _type}

### Run Query

In [ ]:
number_of_results = 10  # You can limit the number of results, pass `None` to fetch all the results

data = forge.search(filters, limit=number_of_results)

print(f"{str(len(data))} dataset(s) of type {_type} found")

### Display the results as pandas dataframe

In [ ]:
property_to_display = ["id","name","subject","brainLocation.brainRegion.id","brainLocation.brainRegion.label","brainLocation.layer.id","brainLocation.layer.label", "contribution","brainLocation.layer.id","brainLocation.layer.label","distribution.name","distribution.contentUrl","distribution.encodingFormat"]
reshaped_data = forge.reshape(data, keep=property_to_display)

forge.as_dataframe(reshaped_data)

### Download

In [ ]:
dirpath = "./downloaded/"
forge.download(data, "distribution.contentUrl", dirpath, overwrite=True)

In [ ]:
ls ./downloaded/

### Display a result as 3D Neuron Morphology

In [ ]:
from neurom import load_neuron
from neurom.view.plotly import draw
import IPython

In [ ]:
neuron = load_neuron(f"{dirpath}/{data[0].distribution.name}")
draw(neuron, inline=False)
IPython.display.HTML(filename='./neuron-3D.html')

## Version the dataset
Tagging a dataset is equivalent to `git tag`. It allows to version a dataset.

In [ ]:
forge.tag(data, value="releaseV112")

In [ ]:
# The version argument can be specified to retrieve the dataset at a given tag.

tagged_data = forge.retrieve(id=data[0].id, version="releaseV112")

In [ ]:
forge.as_dataframe(tagged_data)

In [ ]:
data[0].description="Neuron Morphology from Allen"

In [ ]:
forge.update(data[0])

In [ ]:
non_tagged_data = forge.retrieve(id=data[0].id)

In [ ]:
forge.as_dataframe(non_tagged_data)